confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time
import _init_paths

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib.pyplot import imsave
%matplotlib inline

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [4]:
def evaluate(results):
    
    hist_cumul = 0*results[0]
    for hist in results:
        hist_cumul+=hist
    result = {}
    

    acc = np.diag(hist_cumul).sum() / hist_cumul.sum()
    # per-class accuracy
    pc_acc = np.diag(hist_cumul) / hist_cumul.sum(1)
    mean_accuracy= np.nanmean(pc_acc)
    # per-class IU
    iu = np.diag(hist_cumul) / (hist_cumul.sum(1) + hist_cumul.sum(0) - np.diag(hist_cumul))
    mean_iu= np.nanmean(iu)
    freq = hist_cumul.sum(1) / hist_cumul.sum()
    freq_weighted_iu=(freq[freq > 0] * iu[freq > 0]).sum()
    
    print("Accuracy :",acc)
    print("per-class accuracy :",mean_accuracy)
    print("per-class IU :",mean_iu)
    print("per-class IU (frequency weighted) :",freq_weighted_iu)
    
    return acc

In [5]:
ICIPmodels = '/home/jogue/workspace/ICIPmodels/'
prototxt_path = ICIPmodels+'fuse_test_JGbackend.prototxt'
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [6]:
net = caffe.Net(prototxt_path, caffe.TEST)
#net = caffe.Net(prototxt_path,weights_path , caffe.TEST)

Number of image : 654
Separated in 37 classes : {'toilet': 32, 'whiteboard': 29, 'wall': 0, 'night_stand': 31, 'books': 22, 'mirror': 18, 'table': 6, 'chair': 4, 'floor': 1, 'tv': 24, 'lamp': 34, 'window': 8, 'curtain': 15, 'blinds': 12, 'dresser': 16, 'picture': 10, 'ceiling': 21, 'door': 7, 'shower_curtain': 27, 'fridge': 23, 'cabinet': 2, 'sink': 33, 'desk': 13, 'bookshelf': 9, 'towel': 26, 'box': 28, 'floor_mat': 19, 'shelves': 14, 'sofa': 5, 'counter': 11, 'bed': 3, 'person': 30, 'paper': 25, 'bag': 36, 'bathtub': 35, 'pillow': 17, 'clothes': 20}


#### Updating with new params

In [7]:
Cases = {}

Cases['RGB10-100_DHA_fuseTrained']={}
Cases['RGB10-100_DHA_fuseTrained']['weights'] = '/home/jogue/workspace/ICIPmodels/weights/fuse_weights_RGB10-100_DHA.caffemodel'
#Cases['RGB10-100_DHA_fuseTrained']['dataset'] = '/home/jogue/workspace/datasets/SUNRGBD_pv/data/rgb_i_100_8bits/'
Cases['RGB10-100_DHA_fuseTrained']['dataset_'] = '/home/jogue/workspace/datasets/SUNRGBD_pv/data/d_raw_DHA_8bits/'

Cases['RGB10-100_DHA']={}
Cases['RGB10-100_DHA']['weights'] = '/home/jogue/workspace/ICIPmodels/weights/fuse_weights_RGB10-100_DHA.caffemodel'
#Cases['RGB10-100_DHA']['dataset'] = '/home/jogue/workspace/datasets/SUNRGBD_pv/data/rgb_i_100_8bits/'
Cases['RGB10-100_DHA']['dataset_'] = '/home/jogue/workspace/datasets/SUNRGBD_pv/data/d_raw_DHA_8bits/'


In [56]:
range(100,0,-20)

[100, 80, 60, 40, 20]

In [57]:
imagesToRender = range(1,650,50)
#imagesToRender = [0]
for case in log_progress(Cases.keys(),title='Model', end_msg='Finished !'): 
        weights_path = Cases[case]['weights']
        #net.copy_from(weights_path)
        for luminosity_datatest in log_progress(range(100,0,-20), title='Luminosity dataset', removeAtTheEnd=True):
        for luminosity_datatest in log_progress(range(100,0,-20), title='Luminosity dataset', removeAtTheEnd=True):
            newParams = {}
            newParams['images_folder']= '/home/jogue/workspace/datasets/SUNRGBD_pv/data/rgb_i_'+str(luminosity_datatest)+'_8bits/'
            newParams['images_folder_']= Cases[case]['dataset_']
            newParams['']= Cases[case]['dataset_']
            newParams['iter_counter']=0
            newParams['countEpoch']=0
            net.layers[0].update(newParams)


            resultsConcat = []
            resultsRC = []
            resultsSum = []
            resultsData0 = []
            resultsData1 = []

            lossesConcat = []
            lossesRC = []
            lossesSum = []
            lossesData0 = []
            lossesData1 = []

            n_cl = net.blobs['final_score_rc'].channels
            hist = np.zeros((n_cl, n_cl))
            loss = 0
            print("###############################################################")
            print("NEW TESTING : model",case)
            for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg=str(i),removeAtTheEnd=True):
            #for idx in log_progress(imagesToRender, title='iteration', removeAtTheEnd=True):
                newParams = {}
                newParams['iter_counter']=idx
                net.layers[0].update(newParams)
                print("Idx :",idx)
                net.forward()
                
                
                data0 = net.blobs['data0'].data[0]
                data1 = net.blobs['data1'].data[0]
                
            
                data0 = data0.transpose((1,2,0))
                data0 = data0[:,:,::-1]
                data0 = data0[:,:,::-1]
                #                 data0 = data0[:,:,::-1]
                data1 = data1.transpose((1,2,0))
                data1 = data1[:,:,::-1]
                data1 = data1[:,:,::-1]
                #                 data1 = data1[:,:,::-1]
                
                final_score_rc = net.blobs['final_score_rc'].data[0].argmax(0)
                final_score_concat = net.blobs['final_score_concat'].data[0].argmax(0)
                final_score_sum = net.blobs['final_score_sum'].data[0].argmax(0)
                score_data1 = net.blobs['score_data1'].data[0].argmax(0)
                score_data0 = net.blobs['score_data0'].data[0].argmax(0)


                gt_map = net.blobs['label'].data[0, 0]
                histfinal_score_rc = fast_hist(gt_map.flatten(),final_score_rc.flatten(),n_cl)
                histfinal_score_concat = fast_hist(gt_map.flatten(),final_score_concat.flatten(),n_cl)
                histfinal_score_sum = fast_hist(gt_map.flatten(),final_score_sum.flatten(),n_cl)
                histscore_data1 = fast_hist(gt_map.flatten(),score_data1.flatten(),n_cl)
                histscore_data0 = fast_hist(gt_map.flatten(),score_data0.flatten(),n_cl)
                # compute the loss as well

                loss_data0 = net.blobs['loss_data0'].data.flat[0]
                loss_data1 = net.blobs['loss_data1'].data.flat[0]
                loss_sum = net.blobs['loss_sum'].data.flat[0]
                loss_concat = net.blobs['loss_concat'].data.flat[0]
                loss_rc = net.blobs['loss_rc'].data.flat[0]

                resultsConcat.append(histfinal_score_concat)
                resultsRC.append(histfinal_score_rc)
                resultsSum.append(histfinal_score_sum)
                resultsData0.append(histscore_data0)
                resultsData1.append(histscore_data1)

                lossesConcat.append(loss_concat)
                lossesRC.append(loss_rc)
                lossesSum.append(loss_sum)
                lossesData0.append(loss_data0)
                lossesData1.append(loss_data1)
                
                
                
                pathImg = '{}{}.{}'.format(net.layers[0].images_folder, net.layers[0].list_images[net.layers[0].permutation[idx]],net.layers[0].image_file_extension)
                a = imread(pathImg)
            
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_data0_'+str(luminosity_datatest)+'.png',a)
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_data1.png',data1)
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_final_score_sum_'+str(luminosity_datatest)+'.png',final_score_sum)
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_score_data0_'+str(luminosity_datatest)+'.png',score_data0)
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_score_data1.png',score_data1)
                imsave('/home/jogue/workspace/datasets/SUNRGBD_pv/outputs/'+str(idx+1)+'_gt_map.seg.png',gt_map)
            print('data0')
            evaluate(resultsData0)
            print('data1')
            evaluate(resultsData1)
            print('sum')
            evaluate(resultsSum)
            print('Concat')
            evaluate(resultsConcat)
            print('RC')
            evaluate(resultsRC)

###############################################################
NEW TESTING : model RGB10-100_DHA
Idx : 1
Idx : 51
Idx : 101
Idx : 151
Idx : 201
Idx : 251
Idx : 301
Idx : 351
Idx : 401
Idx : 451
Idx : 501
Idx : 551
Idx : 601
data0
Accuracy : 0.677228567093
per-class accuracy : 0.456989597794
per-class IU : 0.287754284854
per-class IU (frequency weighted) : 0.543113658636
data1
Accuracy : 0.659911281664
per-class accuracy : 0.375121277313
per-class IU : 0.247705945311
per-class IU (frequency weighted) : 0.52832536549
sum
Accuracy : 0.725676606489
per-class accuracy : 0.461499371571
per-class IU : 0.338702565577
per-class IU (frequency weighted) : 0.593667922381
Concat
Accuracy : 0.280590148221
per-class accuracy : 0.0357142857143
per-class IU : 0.0100210767222
per-class IU (frequency weighted) : 0.0787308312784
RC
Accuracy : 0.637393196037
per-class accuracy : 0.357735883406
per-class IU : 0.226103021648
per-class IU (frequency weighted) : 0.535267451437
################################

/home/jogue/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide
/home/jogue/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in true_divide


Idx : 51
Idx : 101
Idx : 151
Idx : 201
Idx : 251
Idx : 301
Idx : 351
Idx : 401
Idx : 451
Idx : 501
Idx : 551
Idx : 601
data0
Accuracy : 0.674681261293
per-class accuracy : 0.434999761307
per-class IU : 0.273128327654
per-class IU (frequency weighted) : 0.540311765206
data1
Accuracy : 0.659911281664
per-class accuracy : 0.375121277313
per-class IU : 0.247705945311
per-class IU (frequency weighted) : 0.52832536549
sum
Accuracy : 0.738414356048
per-class accuracy : 0.446227342514
per-class IU : 0.33202053049
per-class IU (frequency weighted) : 0.603430247069
Concat
Accuracy : 0.280590148221
per-class accuracy : 0.0357142857143
per-class IU : 0.0100210767222
per-class IU (frequency weighted) : 0.0787308312784
RC
Accuracy : 0.635232807313
per-class accuracy : 0.347270080588
per-class IU : 0.217490238731
per-class IU (frequency weighted) : 0.533858618327
###############################################################
NEW TESTING : model RGB10-100_DHA
Idx : 1
Idx : 51
Idx : 101
Idx : 151
Idx 

In [44]:
data1.shape

(427, 561, 3)

In [28]:
a = np.zeros((480,640,3))
a[:,:,::-1].shape

(480, 640, 3)